In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/irina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import re
import os
import warnings

import sys

import numpy as np
import pandas as pd
from pymystem3 import Mystem
from nltk.corpus import stopwords

In [4]:
mystem = Mystem()
swords = set(stopwords.words('russian'))

In [5]:
from text_preprocess import preprocess_tg
files = os.listdir('Telegram_data')
if '.ipynb_checkpoints' in files:
    files.remove('.ipynb_checkpoints')
c = 1

for file in files:
    print(f'file {file} in work')
    warnings.filterwarnings('ignore')
    df = preprocess_tg(f'/{file}', swords)  # Передаем swords
    warnings.filterwarnings('default')
    
    if df is not None:  # Проверяем, что df не None
        df.to_csv(f'Telegram_prep/{file[:-5]}.csv', index=False)  
        print(f'{c} из {len(files)} готов')
        c += 1
    else:
        print(f'Ошибка при обработке файла {file}. Результат: None.')

file newssmartlab_messages.json in work
message ready
1 из 25 готов
file rt_russian_messages.json in work
message ready
2 из 25 готов
file SberInvestments_messages.json in work
message ready
3 из 25 готов
file Stock_News100_messages.json in work
message ready
4 из 25 готов
file frank_media_messages.json in work
message ready
5 из 25 готов
file rian_ru_messages.json in work
message ready
6 из 25 готов
file banki_economy_messages.json in work
message ready
7 из 25 готов
file economylive_messages.json in work
message ready
8 из 25 готов
file gazprom_messages.json in work
message ready
9 из 25 готов
file investingcorp_messages.json in work
message ready
10 из 25 готов
file tass_agency_messages.json in work
message ready
11 из 25 готов
file rbc_news_messages.json in work
message ready
12 из 25 готов
file nedvizhna24_messages.json in work
message ready
13 из 25 готов
file gazpromneft_official_messages.json in work
message ready
14 из 25 готов
file bankser_messages.json in work
message ready


In [1]:
from text_preprocess import telegram_to_standard_date
telegram_to_standard_date()

Обрабатывается файл rt_russian_messages.csv
Обрабатывается файл econs_messages.csv
Обрабатывается файл banki_economy_messages.csv
Обрабатывается файл SberInvestments_messages.csv
Обрабатывается файл Stock_News100_messages.csv
Обрабатывается файл kommersant_messages.csv
Обрабатывается файл headlines_for_traders_messages.csv
Обрабатывается файл markettwits_messages.csv
Обрабатывается файл forbesrussia_messages.csv
Обрабатывается файл investingcorp_messages.csv
Обрабатывается файл banksta_messages.csv
Обрабатывается файл rian_ru_messages.csv


/Users/irina/Desktop/Master_VKR/code/price_prediction_news/src/text_preprocess.py:165: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'Telegram_prep/{file}')


Обрабатывается файл economylive_messages.csv
Обрабатывается файл sosisochniyparserru_messages.csv
Обрабатывается файл newssmartlab_messages.csv
Обрабатывается файл gazpromneft_official_messages.csv
Обрабатывается файл tass_agency_messages.csv


/Users/irina/Desktop/Master_VKR/code/price_prediction_news/src/text_preprocess.py:165: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'Telegram_prep/{file}')


Обрабатывается файл BIoomberg_messages.csv
Обрабатывается файл vedomosti_messages.csv
Обрабатывается файл nedvizhna24_messages.csv
Обрабатывается файл rbc_news_messages.csv
Обрабатывается файл gazprom_messages.csv
Обрабатывается файл frank_media_messages.csv
Обрабатывается файл bankser_messages.csv
Обрабатывается файл FatCat18_messages.csv


### Формирование датасета

In [7]:
companies = {
    'VTBR': 'vtb', 
   'GAZP' : 'gazprom', 
    'SBER': 'sberbank',
    'NVTK': 'novatek',
    'ROSN': 'rosneft'}
data_dir = 'finam_data'

In [15]:
output_dir = 'Stock_data'
# Список компаний
compani = ['SBER', 'GAZP', 'VTBR', 'NVTK', 'ROSN'] 

# Обработка каждой компании
for company_name in compani:
    # Список файлов для текущей компании
    files = [f for f in os.listdir(data_dir) if f.startswith(company_name)]

    # Объединение данных
    data = pd.DataFrame()

    for file in files:
        file_path = os.path.join(data_dir, file)
        df = pd.read_csv(file_path)
        data = pd.concat([data, df], ignore_index=True)

    # Сохранение результата
    output_file = f'{company_name}_20200101_20250331.csv'
    output_path = os.path.join(output_dir, output_file)
    data.to_csv(output_path, index=False)

    print(f'Файл {output_file} успешно создан.')

Файл SBER_20200101_20250331.csv успешно создан.
Файл GAZP_20200101_20250331.csv успешно создан.
Файл VTBR_20200101_20250331.csv успешно создан.
Файл NVTK_20200101_20250331.csv успешно создан.
Файл ROSN_20200101_20250331.csv успешно создан.


In [49]:
#Собираем данные дат в один файл
folders = os.listdir('Stock_data')
folders = [f for f in folders if f[0] != '.']
for folder in folders:
    for company in companies:
        files = os.listdir(f'Stock_data/{folder}')
        files = [f for f in files if company in f and '0101' in f]
        data = pd.DataFrame()
        for file in files:
            try:
                tmp = pd.read_csv(f'Stock_data/{folder}/{file}', sep=';')[['<DATE>', '<TIME>', '<CLOSE>']]
                if folder == '1 день':
                    time = tmp['<DATE>'].astype(str)
                    tmp['timestamp'] = pd.to_datetime(time, format='%y%m%d')
                    tmp.sort_values('timestamp', inplace=True)
                elif folder == '1 час':
                    time = tmp['<DATE>'].astype(str) + tmp['<TIME>'].astype(str).map(lambda x: '235000' if x == '0' else x)
                    tmp['timestamp'] = pd.to_datetime(time, format='%y%m%d%H%M%S')
                    tmp.sort_values('timestamp', inplace=True)
                else:
                    time = tmp['<DATE>'].astype(str) + tmp['<TIME>'].astype(str).map(lambda x: '235000' if x == '0' else x)
                    tmp['timestamp'] = pd.to_datetime(time, format='%y%m%d%H%M%S')
                    tmp.sort_values('timestamp', inplace=True)

                tmp.drop(columns=['<DATE>', '<TIME>'], inplace=True)
                tmp.columns = ['close', 'timestamp']
                tmp.sort_values('timestamp', inplace=True)
                data = pd.concat([data, tmp], axis=0)
            except Exception as ex:
                print(folder)
                print(file)
                print(ex)
        data.to_csv(f'Stock_data/{folder}/{company}.csv', index=False)
        print(f'{company} готов')

SBER готов
GAZP готов
VTBR готов
NVTK готов
ROSN готов


In [7]:
from text_preprocess import add_target_tg
add_target_tg(companies)

VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов


/Users/irina/Desktop/Master_VKR/code/price_prediction_news/src/text_preprocess.py:104: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'Telegram_prep/{file}')


VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов


/Users/irina/Desktop/Master_VKR/code/price_prediction_news/src/text_preprocess.py:104: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'Telegram_prep/{file}')


VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов
VTBR готов
GAZP готов
SBER готов
NVTK готов
ROSN готов


In [7]:
#??? нужны ли бинанрные даные
from text_preprocess import add_target_bin_tg
add_target_bin_tg(companies)

FileNotFoundError: [Errno 2] No such file or directory: '/Stock_data/1 мин./VTBR.csv'

Принадлежность новости к компании

In [9]:
for_regex = {
    'VTBR': ['втб', 'почта банк', 'транскредитбанк', 'vtb', 'банк москвы', 'мультикарта', 'галс девелопмент',
            'бм банк', 'согаз'],
    'GAZP': ['газпром', 'gazp', 'северный поток', 'сила сибири', 'голубой поток', 'ямал европа',
             'ямал европа', 'ухта торжок', 'gazprom'], 
    'SBER': ['/^сбер$/', 'сбербанк', 'sber', 'сбермегамаркет', 'сбермаркет', 'сберздоровье'],
    'NVTK': ['новатэк', 'nvtk', 'novatek', 'ямал спг', 'арктикгаз', 'нортгаз', 'арктик спг', 'транснефтегаз'],
    'ROSN': ['роснефть', 'rosn', 'самаранефтегаз', 'rosneft', 'роснефти']
}

In [9]:
from text_preprocess import add_company_tg
add_company_tg(for_regex, companies)

markettwits.csv done


/Users/irina/Desktop/Master_VKR/code/price_prediction_news/src/text_preprocess.py:58: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'Telegram_prep/{file}')


rian_ru.csv done
Stock_News100.csv done
bankser.csv done
rt_russian_messages.csv done
econs_messages.csv done
banki_economy_messages.csv done
economylive.csv done
tass_agency.csv done
SberInvestments_messages.csv done
FatCat18.csv done
Stock_News100_messages.csv done
kommersant_messages.csv done
rt_russian.csv done
banki_economy.csv done
forbesrussia.csv done
kommersant.csv done
headlines_for_traders_messages.csv done
nedvizhna24.csv done
markettwits_messages.csv done
forbesrussia_messages.csv done
investingcorp_messages.csv done
BIoomberg.csv done
gazpromneft_official.csv done
banksta_messages.csv done
SberInvestments.csv done


/Users/irina/Desktop/Master_VKR/code/price_prediction_news/src/text_preprocess.py:58: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'Telegram_prep/{file}')


rian_ru_messages.csv done
economylive_messages.csv done
newssmartlab.csv done
sosisochniyparserru_messages.csv done
newssmartlab_messages.csv done
gazpromneft_official_messages.csv done
investingcorp.csv done


/Users/irina/Desktop/Master_VKR/code/price_prediction_news/src/text_preprocess.py:58: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'Telegram_prep/{file}')


tass_agency_messages.csv done
gazprom.csv done
sosisochniyparserru.csv done
vedomosti.csv done
banksta.csv done
BIoomberg_messages.csv done
econs.csv done
vedomosti_messages.csv done
nedvizhna24_messages.csv done
rbc_news_messages.csv done
gazprom_messages.csv done
rbc_news.csv done
frank_media_messages.csv done
bankser_messages.csv done
headlines_for_traders.csv done
FatCat18_messages.csv done
frank_media.csv done


In [11]:
tg_root = 'Telegram_prep'
import os
folders = os.listdir(tg_root)
folders = [f for f in folders if 'csv' not in f]

In [13]:
### Телеграмм
agg_df = pd.DataFrame()
if '.ipynb_checkpoints' in files:
        files.remove('.ipynb_checkpoints')
for folder in folders:
    files = [
        folder + el + '.parquet' for el in ['_data', '_abs_values', '_rel_values', '_company']
    ]
    tmp_agg_df = pd.DataFrame()
    for file in files:
        tmp = pd.read_parquet(tg_root + '/' + folder + '/' + file)
        tmp_agg_df = pd.concat([tmp_agg_df, tmp], axis=1)
    tmp_agg_df['source'] = folder
    agg_df = pd.concat([agg_df, tmp_agg_df])
    print(folder)
agg_df.to_parquet('Data/tg.parquet', index=False)

NameError: name 'files' is not defined